In [1]:
using Revise

using RigidBodyDynamics
using RigidBodyDynamics: Bounds

using RigidBodyTreeInspector
using DrakeVisualizer
using Plots

using BilevelTrajOpt

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/8c8a043c2f00369f3e7ae550277d2a54e59a34ad-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/8c8a043c2f00369f3e7ae550277d2a54e59a34ad-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [2]:
urdf = joinpath("..", "urdf", "ball.urdf")
mechanism = parse_urdf(Float64, urdf)
body = findbody(mechanism, "ball")
basejoint = joint_to_parent(body, mechanism)
floatingjoint = Joint(basejoint.name, frame_before(basejoint), frame_after(basejoint), QuaternionFloating{Float64}())
replace_joint!(mechanism, basejoint, floatingjoint)
position_bounds(floatingjoint) .= Bounds(-100, 100)
velocity_bounds(floatingjoint) .= Bounds(-100, 100)
μ = 0.5
motion_type = :xyz
point = Point3D(default_frame(findbody(mechanism, "floor")), [0.,0.,0.])
normal = FreeVector3D(default_frame(findbody(mechanism, "floor")), [0.,0.,1.])
hs = HalfSpace(point, normal)
floor = Obstacle(hs, μ, motion_type)
obstacles = [floor]
env = parse_contacts(mechanism, urdf, obstacles)
x0 = MechanismState(mechanism)
Δt = 0.005
sim_data = get_sim_data(x0,env,Δt,true);

In [17]:
q0 = [1., 0., 0., 0., 0., 0., 1.]
v0 = [0., 0., 0., 1., 1., 0.]
u0 = zeros(sim_data.num_v)
set_configuration!(x0, q0)
set_velocity!(x0, v0)
setdirty!(x0)
ctrl! = (u,t,x) -> u[:] = 0.
traj = BilevelTrajOpt.simulate(x0,env,sim_data.Δt,1,ctrl!,implicit_contact=false)
qnext = traj[1:sim_data.num_q,2]
vnext = traj[sim_data.num_q+1:sim_data.num_q+sim_data.num_v,2]
x_sol_exp = traj[sim_data.num_q+sim_data.num_v+2:end,2]

τ_ip, x_sol_ip = solve_implicit_contact_τ(sim_data,q0,v0,u0,qnext,vnext,ip_method=true);

τ_auglag, x_sol_auglag = solve_implicit_contact_τ(sim_data,q0,v0,u0,qnext,vnext,ip_method=false);

display("Explicit")
display(x_sol_exp)
display("IP")
display(x_sol_ip)
display("Aug Lag")
display(x_sol_auglag)

"Explicit"

6-element Array{Float64,1}:
 -7.76876e-10
 -7.76876e-10
 -4.93409e-10
 -4.93409e-10
  8.30333    
 -6.3857e-9  

"IP"

6-element Array{Float64,1}:
 1.44162e-9
 1.44162e-9
 1.44298e-9
 1.44298e-9
 1.0       
 1.44329e-8

"Aug Lag"

6-element Array{Float64,1}:
 -5.22879e-13
 -5.2288e-13 
  4.06108e-13
  4.06111e-13
  1.0        
 -9.974e-13  

Solve_Succeeded
Solve_Succeeded


In [ ]:
ForwardDiff.jacobian(x̃ -> solve_implicit_contact_τ(sim_data,q0,v0,u0,x̃[1:sim_data.num_q],x̃[sim_data.num_q+1:sim_data.num_q+sim_data.num_v];ip_method=false)[1],vcat(qnext,vnext))